[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adaptive-intelligent-robotics/QDax/blob/main/examples/mapelites.ipynb)

# Optimizing with Occupancy-CVT MAP-Elites in JAX

This notebook shows how to use QDax to find diverse and performing controllers using the [occupancy measure as a generic behavioural descriptor](https://dl.acm.org/doi/10.1145/3712256.3726337).
It can be run locally or on Google Colab. We recommend to use a GPU.

This notebook uses code from the QDax notebook on CVT MAP-Elites	and Policy Gradient Assisted MAP-Elites (PGA-ME) for those elements of the process that remain unchanged. The main contribution is the setup of the occupancy measure as a behavioural descriptor that utilised a full rollout as data. It will show:

- the usual PGA-MAP-Elites setup
- how to create the behavioural descriptor using the double CVT-trick
- how to create the centroids in the occupancy space
- an example solving the Point-Maze and some evaluation

In [ ]:
from IPython.display import clear_output

try:
    import qdax
except:
    print("QDax not found. Installing...")
    !pip install qdax[cuda12]
    import qdax

clear_output()

In [ ]:
#brax version 0.12.4 has changed the API
!pip install brax==0.12.3

In [ ]:
!pip install ipympl | tail -n 1
# %matplotlib widget
# from google.colab import output
# output.enable_custom_widget_manager()

import os

from IPython.display import clear_output
import functools
import time

import jax
import jax.numpy as jnp

import numpy as np
from numpy.random import RandomState
from sklearn.cluster import KMeans

from qdax.core.map_elites import MAPElites
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids, MapElitesRepertoire
import qdax.tasks.brax.v1 as environments
from qdax.tasks.brax.v1.env_creators import scoring_function_brax_envs as scoring_function
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.networks.networks import MLP
from qdax.core.emitters.mutation_operators import isoline_variation
from qdax.core.emitters.standard_emitters import MixingEmitter
from qdax.core.emitters.pga_me_emitter import PGAMEConfig, PGAMEEmitter
#from qdax.utils.plotting import plot_map_elites_results

from qdax.utils.metrics import CSVLogger, default_qd_metrics

from jax.flatten_util import ravel_pytree

#from IPython.display import HTML
#from brax.v1.io import html


if "COLAB_TPU_ADDR" in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

clear_output()

In [ ]:
#@title QD Training Definitions Fields
#@markdown ---
batch_size = 100 #@param {type:"number"}
env_name = 'pointmaze'#@param['pointmaze', 'anttrap', 'walker2d_uni']
episode_length = 200 #@param {type:"integer"}
num_iterations = 2000 #@param {type:"integer"}
seed = 42 #@param {type:"integer"}
policy_hidden_layer_sizes = (64, 64) #@param {type:"raw"}
iso_sigma = 0.005 #@param {type:"number"}
line_sigma = 0.05 #@param {type:"number"}
num_init_cvt_samples = 50000 #@param {type:"integer"}
num_centroids_stateaction = 512 #@param {type:"integer"}
num_centroids_repertoire = 512 #@param {type:"integer"}
min_descriptor = -1.0 #@param {type:"number"}
max_descriptor = 1.0 #@param {type:"number"}
#@markdown ---

## The standard MAP-Elites setup

This follows the notebook for CVT-Elites:

Define the environment in which the policies will be trained. In this notebook, we focus on controllers learning to move a robot in a physical simulation. We also define the shared policy, that every individual in the population will use. Once the policy is defined, all individuals are defined by their parameters, that corresponds to their genotype.

In [ ]:
# Init environment
env = environments.create(env_name, episode_length=episode_length)
reset_fn = jax.jit(env.reset)

# Init a random key
key = jax.random.key(seed)

# Init policy network
policy_layer_sizes = policy_hidden_layer_sizes + (env.action_size,)
policy_network = MLP(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)

# Init population of controllers
key, subkey = jax.random.split(key)
keys = jax.random.split(subkey, num=batch_size)
fake_batch = jnp.zeros(shape=(batch_size, env.observation_size))
init_variables = jax.vmap(policy_network.init)(keys, fake_batch)

Now that the environment and policy has been defined, it is necessary to define a function that describes how the policy must be used to interact with the environment and to store transition data.

In [ ]:
# Define the function to play a step with the policy in the environment
def play_step_fn(
    env_state,
    policy_params,
    key,
):
    """
    Play an environment step and return the updated state and the transition.
    """
    actions = policy_network.apply(policy_params, env_state.obs)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, actions)
    transition = QDTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        actions=actions,
        truncations=next_state.info["truncation"],
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
    )

    return next_state, policy_params, key, transition

The emitter is used to evolve the population at each mutation step. This follows the PGA-MAP-Elites setup.

In [ ]:
###### PG Parameters
proportion_mutation_ga = 0.5  # @param {type:"number"}
# TD3 params
replay_buffer_size = 1000000  # @param {type:"number"}
critic_hidden_layer_size = policy_hidden_layer_sizes  # @param {type:"raw"}
critic_learning_rate = 3e-4  # @param {type:"number"}
greedy_learning_rate = 3e-4  # @param {type:"number"}
policy_learning_rate = 1e-3  # @param {type:"number"}
noise_clip = 0.5  # @param {type:"number"}
policy_noise = 0.2  # @param {type:"number"}
discount = 0.99  # @param {type:"number"}
reward_scaling = 1.0  # @param {type:"number"}
transitions_batch_size = 256  # @param {type:"number"}
soft_tau_update = 0.005  # @param {type:"number"}
num_critic_training_steps = 300  # @param {type:"number"}
num_pg_training_steps = 100  # @param {type:"number"}
policy_delay = 2  # @param {type:"number"}
# @markdown ---
# Define the PG-emitter config
pga_emitter_config = PGAMEConfig(
    env_batch_size=batch_size,
    batch_size=transitions_batch_size,
    proportion_mutation_ga=proportion_mutation_ga,
    critic_hidden_layer_size=critic_hidden_layer_size,
    critic_learning_rate=critic_learning_rate,
    greedy_learning_rate=greedy_learning_rate,
    policy_learning_rate=policy_learning_rate,
    noise_clip=noise_clip,
    policy_noise=policy_noise,
    discount=discount,
    reward_scaling=reward_scaling,
    replay_buffer_size=replay_buffer_size,
    soft_tau_update=soft_tau_update,
    num_critic_training_steps=num_critic_training_steps,
    num_pg_training_steps=num_pg_training_steps,
    policy_delay=policy_delay,
)
# Define emitter
variation_fn = functools.partial(isoline_variation, iso_sigma=0.05, line_sigma=0.1)
pg_emitter = PGAMEEmitter(
    config=pga_emitter_config,
    policy_network=policy_network,
    env=env,
    variation_fn=variation_fn,
)


## Occupancy CVT-Elites

This is the minimal code to run the double CVT-trick to use the occupancy measure as a behavioural descriptor. Here, define functions (using JAX-logic) that 1) create a behavioural descriptor based on a CVT of the state-action space and another CVT of the occupancy space, and 2) create a CVT of the occupancy space.

The method needs two CVTs: one standard CVT for the state-action space and another for the occupancy space. This space is not a "box" (i.e. [a_1, b_1] x ... x [a_n, b_n]) but rather a space of 0 = x_i = 1 such that sum(x_i) = 1.
The behavioural descriptor takes an episode, ignores all data that is gathered after the episode finishes, measures the visitation of the "bins" of the state-action space according to the first CVT and assigns this measure to a bin in the second CVT.

In [ ]:
def get_distr(neighbors, valid_steps, num_centroids):
    # Set non-valid (steps that have been taken after the episode terminated) as "unreachable"/"uncountable"
    idx = jnp.maximum(jnp.arange(num_centroids), valid_steps)
    neighbors = neighbors.at[idx].set(
        num_centroids
    )
    # Count how often one "bin" is visited
    return jnp.bincount(neighbors.flatten(), length=num_centroids).astype(float)

# fast calculation of nearest neighbours
@functools.partial(jax.jit, static_argnames=["num_centroids", "recall_target"])
def get_neighbors(centroids, centroids_sq, y, num_centroids, recall_target):
    y_square = jnp.sum(y**2, axis=1, keepdims=True)
    dists = centroids_sq + y_square.T * jnp.dot(centroids, y.T)
    neighbor_dists, neighbors = jax.lax.approx_min_k(
        dists.T, k=1, recall_target=recall_target
    )
    return neighbors

def bd_distribution_fn_withcentroids(
    data: QDTransition,
    mask: jnp.ndarray,
    ft_centroids: jnp.ndarray,
    ft_centroids_sq: jnp.ndarray,
    num_centroids: int,
):
    # tanh is a lazy way of making all observation data bounded in [-1,1]
    # concat obs and action to get the full state-action space
    inp = jnp.tanh(jnp.concat([data.obs, data.actions], axis=-1))

    # find nearest neighbours to assign them to the appropriate bin
    all_neighbors = jax.vmap(get_neighbors, in_axes=(None, None, 0, None, None))(
        ft_centroids, ft_centroids_sq, inp, num_centroids, 0.95
    )
    # find non-terminal steps
    all_valid_steps = (1 - mask).sum(axis=1).astype(int)

    # only use those, that are non-terminal
    nns = jax.vmap(get_distr, in_axes=(0, 0, None))(
        all_neighbors, all_valid_steps, num_centroids
    )
    # normalise to sum == 1
    dists = nns / jnp.expand_dims(all_valid_steps, axis=1)
    return dists


def compute_simplex_centroids(
    num_descriptors,
    num_init_cvt_samples,
    num_centroids,
    key,
):
    # since all samples are x_i > 0 and sum(x_i) == 1, sample from the direchtlet distribution
    x = jax.random.dirichlet(
        key, jnp.ones(num_descriptors), shape=(num_init_cvt_samples,)
    )
    k_means = KMeans(
        init="k-means++",
        n_clusters=num_centroids,
        n_init=1,
        random_state=RandomState(42),
    )
    k_means.fit(x)
    return jnp.asarray(k_means.cluster_centers_)

Now, create the CVT and the behavioural descriptor function.

In [ ]:
# Compute the centroids for the state-action space (1st CVT)
key, subkey = jax.random.split(key)
ft_centroids = compute_cvt_centroids(
    num_descriptors=env.observation_size + env.action_size,
    num_init_cvt_samples=num_init_cvt_samples,
    num_centroids=num_centroids_repertoire,
    minval=-1,
    maxval=1,
    key=subkey,
)

# calculate this square sum once only
ft_centroids_sq = jnp.sum(ft_centroids**2, axis=1, keepdims=True)
bd_distribution = functools.partial(
    bd_distribution_fn_withcentroids,
    ft_centroids=ft_centroids,
    ft_centroids_sq=ft_centroids_sq,
    num_centroids=num_centroids_repertoire,
)

In [ ]:
# Compute the centroids for the occupancy measure space (2nd CVT)
key, subkey = jax.random.split(key)
elites_centroids = compute_simplex_centroids(
    num_centroids_repertoire,
    num_init_cvt_samples,
    num_centroids_stateaction,
    subkey,
)

Now that the behavioural descriptor function `bd_distribution` is defined, the scoring function can be defined.

The scoring function is used in the evaluation step to determine the fitness and descriptor of each individual.

In [ ]:
# Prepare the scoring function
scoring_fn = functools.partial(
    scoring_function,
    episode_length=episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    descriptor_extractor=bd_distribution,
)

# Get minimum reward value to make sure qd_score are positive
reward_offset = environments.reward_offset[env_name]

# Define a metrics function
metrics_function = functools.partial(
    default_qd_metrics,
    qd_offset=reward_offset * episode_length,
)

Now the standard MAPElites logic can be instantiated.

In [ ]:
# Instantiate MAP-Elites
map_elites = MAPElites(
    scoring_function=scoring_fn,
    emitter=pg_emitter,
    metrics_function=metrics_function,
)

# Compute initial repertoire and emitter state
key, subkey = jax.random.split(key)
repertoire, emitter_state, init_metrics = map_elites.init(init_variables, elites_centroids, subkey)

## Launch MAP-Elites iterations

In [ ]:
log_period = 10
num_loops = num_iterations // log_period

# Initialize metrics
metrics = {key: jnp.array([]) for key in ["iteration", "qd_score", "coverage", "max_fitness", "time"]}

# Set up init metrics
init_metrics = jax.tree.map(lambda x: jnp.array([x]) if x.shape == () else x, init_metrics)
init_metrics["iteration"] = jnp.array([0], dtype=jnp.int32)
init_metrics["time"] = jnp.array([0.0])  # No time recorded for initialization

# Convert init_metrics to match the metrics dictionary structure
metrics = jax.tree.map(lambda metric, init_metric: jnp.concatenate([metric, init_metric], axis=0), metrics, init_metrics)

# Initialize CSV logger
csv_logger = CSVLogger(
    "mapelites-logs.csv",
    header=list(metrics.keys())
)

# Log initial metrics
csv_logger.log(jax.tree.map(lambda x: x[-1], metrics))

# Main loop
map_elites_scan_update = map_elites.scan_update
for i in range(num_loops):
    start_time = time.time()
    (
        repertoire,
        emitter_state,
        key,
    ), current_metrics = jax.lax.scan(
        map_elites_scan_update,
        (repertoire, emitter_state, key),
        (),
        length=log_period,
    )
    timelapse = time.time() - start_time

    # Metrics
    current_metrics["iteration"] = jnp.arange(1+log_period*i, 1+log_period*(i+1), dtype=jnp.int32)
    current_metrics["time"] = jnp.repeat(timelapse, log_period)
    metrics = jax.tree.map(lambda metric, current_metric: jnp.concatenate([metric, current_metric], axis=0), metrics, current_metrics)

    # Log
    csv_logger.log(jax.tree.map(lambda x: x[-1], metrics))

In [ ]:
#@title Visualization

# Create the x-axis array
env_steps = metrics["iteration"]

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Using only part of plot_map_elites_results from QDax, as the repertoire is not 2D ...
x_label = "Environment steps"
# Customize matplotlib params
font_size = 16
params = {
    "axes.labelsize": font_size,
    "axes.titlesize": font_size,
    "legend.fontsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "text.usetex": False,
    "axes.titlepad": 10,
}

mpl.rcParams.update(params)

# Visualize the training evolution and final repertoire
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(40, 9))

# env_steps = jnp.arange(num_iterations) * episode_length * batch_size

axes[0].plot(env_steps, metrics["coverage"])
axes[0].set_xlabel(x_label)
axes[0].set_ylabel("Coverage in %")
axes[0].set_title("Coverage evolution during training")
axes[0].set_aspect(0.95 / axes[0].get_data_ratio(), adjustable="box")

axes[1].plot(env_steps, metrics["max_fitness"])
axes[1].set_xlabel(x_label)
axes[1].set_ylabel("Maximum fitness")
axes[1].set_title("Maximum fitness evolution during training")
axes[1].set_aspect(0.95 / axes[1].get_data_ratio(), adjustable="box")

axes[2].plot(env_steps, metrics["qd_score"])
axes[2].set_xlabel(x_label)
axes[2].set_ylabel("QD Score")
axes[2].set_title("QD Score evolution during training")
axes[2].set_aspect(0.95 / axes[2].get_data_ratio(), adjustable="box")


In [ ]:
best_idx = jnp.argmax(repertoire.fitnesses)
best_fitness = jnp.max(repertoire.fitnesses)
best_descriptor = repertoire.descriptors[best_idx]

In [ ]:
print(
    f"Best fitness in the repertoire: {best_fitness:.2f}\n",
    f"Index in the repertoire of this individual: {best_idx}\n"
)

In [ ]:
# select the parameters of the best individual
my_params = jax.tree.map(
    lambda x: x[best_idx],
    repertoire.genotypes
)

## Play some steps in the environment

In [ ]:
vis_env = environments.create(env_name, episode_length=episode_length, auto_reset=False)
jit_env_reset = jax.jit(vis_env.reset)
jit_env_step = jax.jit(vis_env.step)
jit_inference_fn = jax.jit(policy_network.apply)

In [ ]:
rollout = []
key, subkey = jax.random.split(key)
state = jit_env_reset(rng=subkey)
rollout.append(state)

while not state.done:
    action = jit_inference_fn(my_params, state.obs)
    state = jit_env_step(state, action)
    rollout.append(state)

print(f"The trajectory of this individual contains {len(rollout)} transitions.")

In [ ]:
episode = [ts.obs for ts in rollout]
episode = np.stack(episode)

At this point there is no built-in way to properly visualise Point-Maze. Here, a simple attempt to show an episode.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_trajectory(data):
    # Hardcoded boundaries and wall positions
    x_min = -1
    x_max = 1
    y_min = -1
    y_max = 1
    upper_wall_height_offset = 0.2
    lower_wall_height_offset = -0.5
    wallheight = 0.01
    wallwidth = (x_max - x_min) * 0.75

    h_zone_width = 0.05
    zone_width_offset = x_min + 0.5
    zone_height_offset = y_max + -0.2

    # Upper wall coordinates
    upper_wall_x_1 = [x_min, x_min + wallwidth]
    upper_wall_y_1 = [upper_wall_height_offset, upper_wall_height_offset]
    upper_wall_x_2 = [x_min, x_min + wallwidth]
    upper_wall_y_2 = [upper_wall_height_offset + wallheight, upper_wall_height_offset + wallheight]

    # Lower wall coordinates
    lower_wall_x_1 = [x_max - wallwidth, x_max]
    lower_wall_y_1 = [lower_wall_height_offset, lower_wall_height_offset]
    lower_wall_x_2 = [x_max - wallwidth, x_max]
    lower_wall_y_2 = [lower_wall_height_offset + wallheight, lower_wall_height_offset + wallheight]

    # Zone
    zone_x_1 = [zone_width_offset - h_zone_width, zone_width_offset + h_zone_width]
    zone_y_1 = [zone_height_offset - h_zone_width, zone_height_offset- h_zone_width]
    zone_x_2 = [zone_width_offset - h_zone_width, zone_width_offset + h_zone_width]
    zone_y_2 = [zone_height_offset + h_zone_width, zone_height_offset + h_zone_width]
    zone_x_3 = [zone_width_offset - h_zone_width, zone_width_offset - h_zone_width]
    zone_y_3 = [zone_height_offset - h_zone_width, zone_height_offset + h_zone_width]
    zone_x_4 = [zone_width_offset + h_zone_width, zone_width_offset + h_zone_width]
    zone_y_4 = [zone_height_offset - h_zone_width, zone_height_offset + h_zone_width]

    fig = plt.figure(figsize=(6, 6))

    # Plotting the trajectory
    plt.plot(data[:, 0], data[:, 1], marker='o', label='Trajectory')

    # Plotting the walls
    plt.plot(upper_wall_x_1, upper_wall_y_1, color='red', label='Wall')
    plt.plot(upper_wall_x_2, upper_wall_y_2, color='red')
    plt.plot(lower_wall_x_1, lower_wall_y_1, color='red')
    plt.plot(lower_wall_x_2, lower_wall_y_2, color='red')

    plt.plot(zone_x_1, zone_y_1, color='green', label = "Goal")
    plt.plot(zone_x_2, zone_y_2, color='green')
    plt.plot(zone_x_3, zone_y_3, color='green')
    plt.plot(zone_x_4, zone_y_4, color='green')

    # Plotting the boundaries
    plt.axhline(y=y_min, color='k', linestyle='--', label='Boundary')
    plt.axhline(y=y_max, color='k', linestyle='--')
    plt.axvline(x=x_min, color='k', linestyle='--')
    plt.axvline(x=x_max, color='k', linestyle='--')

    plt.title('Particle Trajectory in 2D Maze')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.grid()
    plt.axis('equal')
    plt.legend()
    plt.show()

# Example usage:
plot_trajectory(episode)